In [1]:
### TODO:
# 1. Do feature engineering on group level instead of user level
# 2. Separate data between first person mode and free for all mode
# 3. Eliminate cheaters and anomalies
# 4. Develop prediction funcgion
#    - Final ranking per match can be determined using this formula -> 100/maxPlace, as ranking interval

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
from matplotlib import pyplot as plt

%precision %.4f

'%.4f'

In [2]:
### Read training data
train = pd.read_csv('input/train_V2.csv')

### 1. Group Level Inspection and Feature Engineering

Because the ranking is spread based on the number of groups in one match, group level features need to be generated!

In [3]:
### Generate group level features    

def generate_group_level_features(dataset,
                                  feature_columns=['kills','assists','boosts'],
                                  _stats = ['max','min','sum','mean','std']):
    features = dataset[["matchId","groupId",*feature_columns]].reset_index(drop=True)
    matchGroups = features[["matchId","groupId"]].drop_duplicates().reset_index(drop=True)
    
    ### predefined basic statistic operations
    
    ### calculate group level features
    for f in feature_columns:
        for s in _stats:
            new_field = '{s}_{f}'.format(s=s,f=f)
            print(new_field)
            matchGroups = pd.merge(matchGroups,
                features.groupby(["matchId","groupId"],as_index=False)\
                .agg({f:s}).rename(columns={f:new_field}).fillna(0)[["matchId","groupId",new_field]].drop_duplicates(),
                on=['matchId','groupId'],how='inner'
            )
            
    return matchGroups.reset_index(drop=True)

In [4]:
import time
s = time.time()
groupLevelFeatures_train = generate_group_level_features(train, 
                                                         ['boosts', 'damageDealt', 'DBNOs',
                                                          'headshotKills', 'killPlace', 
                                                          'killStreaks', 'longestKill',
                                                          'walkDistance',
                                                          'weaponsAcquired'],
                                                        ['max', 'sum'])
e = time.time()
print("elapsed {}s".format(e-s))

max_boosts
sum_boosts
max_damageDealt
sum_damageDealt
max_DBNOs
sum_DBNOs
max_headshotKills
sum_headshotKills
max_killPlace
sum_killPlace
max_killStreaks
sum_killStreaks
max_longestKill
sum_longestKill
max_walkDistance
sum_walkDistance
max_weaponsAcquired
sum_weaponsAcquired
elapsed 416.2341067790985s


In [5]:
test = pd.read_csv('input/test_V2.csv')

In [6]:
s = time.time()
groupLevelFeatures_test = generate_group_level_features(test, 
                                                         ['boosts', 'damageDealt', 'DBNOs',
                                                          'headshotKills', 'killPlace', 
                                                          'killStreaks', 'longestKill',
                                                          'walkDistance',
                                                          'weaponsAcquired'],
                                                        ['max', 'sum'])
e = time.time()
print("elapsed {}s".format(e-s))

max_boosts
sum_boosts
max_damageDealt
sum_damageDealt
max_DBNOs
sum_DBNOs
max_headshotKills
sum_headshotKills
max_killPlace
sum_killPlace
max_killStreaks
sum_killStreaks
max_longestKill
sum_longestKill
max_walkDistance
sum_walkDistance
max_weaponsAcquired
sum_weaponsAcquired
elapsed 168.06163501739502s


In [7]:
# groupLevelFeatures_train.to_csv("groupLevelFeatures_train.csv",index=False)
del train
del test
groupLevelFeatures_train.head()

,matchId,groupId,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,max_killStreaks,sum_killStreaks,max_longestKill,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired
0,a10357fd1a4a91,4d4b580de459be,0,0,318.0,408.75,2,2,1,1,62,189,1,1,27.66,27.660,342.80,731.96,2,5
1,aeb375fc57110c,684d5656442f9e,3,6,300.8,571.27,2,2,0,0,57,148,2,3,77.97,83.649,2435.00,7571.00,8,24
2,110163d8bb94ae,6a4a42c3245a74,3,3,146.6,214.60,1,1,0,0,47,65,1,1,10.84,10.840,1119.00,1280.80,2,4
3,f1f1f4ef412d7e,a930a9c79cd721,0,0,32.9,32.90,0,0,0,0,75,75,0,0,0.00,0.000,202.70,202.70,3,3
4,6dc8ff871e21e6,de04010b3458dd,0,0,100.0,100.00,0,0,0,0,45,45,1,1,58.53,58.530,49.75,49.75,2,2


In [8]:
groupLevelFeatures_test.head()

,matchId,groupId,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,max_killStreaks,sum_killStreaks,max_longestKill,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired
0,45b576ab7daa7f,676b23c24e70d6,0,0,74.2,125.66,1,1,0,0,73,286,0,0,0.0,0.000,913.0,2436.8,7,10
1,42a9a0b906c928,430933124148dd,6,10,597.5,776.60,4,4,1,1,11,14,2,3,361.9,554.500,2477.0,4494.0,7,13
2,87e7e4477a048e,0b45f5db20ba99,6,9,1058.0,1490.11,5,8,4,4,50,103,2,3,236.8,254.420,2351.0,4397.1,8,15
3,1b9a94f1af67f1,b7497dbdc77f4a,0,0,100.0,165.52,1,1,0,0,54,107,0,0,0.0,0.000,2116.0,3928.0,5,8
4,40754a93016066,6604ce20a1d230,4,7,330.2,661.84,2,4,2,3,23,36,1,3,110.9,173.838,3027.0,8695.0,9,18


In [17]:
# groupLevelFeatures_train.to_csv('input/groupLevelFeatures_train.csv', index=False)
# groupLevelFeatures_test.to_csv('input/groupLevelFeatures_test.csv', index=False)

### 2. Separate Game Modes data

(TODO) <br>
There are several game modes / match types in PUBG <br>
https://pubg.gamepedia.com/Game_Modes <br>
Patterns might differ for example between First Person Mode and Third Person Mode even though the players are on solo queue game.

### 3. Eliminate Anomalies

(TODO) <br>
There are already existing kernel out there mentioning anomalies or cheaters in PUBG matches. <br>
We need to adopt some of them.

### 4. Prediction Functions

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [12]:
train = pd.read_csv('input/train_V2.csv')
train = pd.merge(groupLevelFeatures_train, train[['groupId', 'winPlacePerc']])
# train.sort_values(['matchId', 'winPlacePerc']).head()
train.head()

,matchId,groupId,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,...,sum_killPlace,max_killStreaks,sum_killStreaks,max_longestKill,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired,winPlacePerc
0,a10357fd1a4a91,4d4b580de459be,0,0,318.0,408.75,2,2,1,1,...,189,1,1,27.66,27.660,342.8,731.96,2,5,0.4444
1,a10357fd1a4a91,4d4b580de459be,0,0,318.0,408.75,2,2,1,1,...,189,1,1,27.66,27.660,342.8,731.96,2,5,0.4444
2,a10357fd1a4a91,4d4b580de459be,0,0,318.0,408.75,2,2,1,1,...,189,1,1,27.66,27.660,342.8,731.96,2,5,0.4444
3,a10357fd1a4a91,4d4b580de459be,0,0,318.0,408.75,2,2,1,1,...,189,1,1,27.66,27.660,342.8,731.96,2,5,0.4444
4,aeb375fc57110c,684d5656442f9e,3,6,300.8,571.27,2,2,0,0,...,148,2,3,77.97,83.649,2435.0,7571.00,8,24,0.6400


In [13]:
train = train[train['winPlacePerc'].notnull()].reset_index(drop=True)

In [14]:
# col_metrics = ['assists', 'boosts', 'damageDealt', 'DBNOs',
#        'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
#        'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
#        'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
#        'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
#        'weaponsAcquired', 'winPoints']
# col_drop = ['Id', 'groupId', 'matchId']

# col_metrics = ['boosts', 'damageDealt', 'DBNOs',
#        'headshotKills', 'killPlace', 
#        'killStreaks', 'longestKill',
#        'walkDistance',
#        'weaponsAcquired']
# col_drop = ['Id', 'groupId', 'matchId']
col_drop = ['groupId', 'matchId']

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(train.drop('winPlacePerc', axis=1), 
#                                     train[['groupId', 'winPlacePerc']], test_size=0.33, random_state=42)
X_train = train.drop('winPlacePerc', axis=1)
X_test = groupLevelFeatures_test
y_train = train[['groupId', 'winPlacePerc']]
X_train.shape, X_test.shape, y_train.shape

((4446965, 20), (886238, 20), (4446965, 2))

In [19]:
X_train_2 = X_train.drop(col_drop, axis=1)
X_train_2.head()

,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,max_killStreaks,sum_killStreaks,max_longestKill,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired
0,0,0,318.0,408.75,2,2,1,1,62,189,1,1,27.66,27.660,342.8,731.96,2,5
1,0,0,318.0,408.75,2,2,1,1,62,189,1,1,27.66,27.660,342.8,731.96,2,5
2,0,0,318.0,408.75,2,2,1,1,62,189,1,1,27.66,27.660,342.8,731.96,2,5
3,0,0,318.0,408.75,2,2,1,1,62,189,1,1,27.66,27.660,342.8,731.96,2,5
4,3,6,300.8,571.27,2,2,0,0,57,148,2,3,77.97,83.649,2435.0,7571.00,8,24


In [20]:
# X_train.sort_values(['matchId', 'groupId']).head(20)

# Decision Tree

In [21]:
# classifier = DecisionTreeRegressor(random_state=42)
# classifier.fit(X_train_2, y_train['winPlacePerc'])
# del X_train_2

# Random Forest

In [22]:
classifier = RandomForestRegressor(n_jobs=4, n_estimators=10, random_state=42)
classifier.fit(X_train_2, y_train['winPlacePerc'])
del X_train_2

In [23]:
X_test_2 = X_test.drop(col_drop, axis=1)
y_pred = classifier.predict(X_test_2)
del X_test_2
y_pred, X_test.shape, len(y_pred)

(array([0.26256   , 0.91882333, 0.84822   , ..., 0.19542   , 0.46775   ,
        0.009487  ]), (886238, 20), 886238)

In [24]:
# mean_absolute_error(y_test['winPlacePerc'], y_pred)

In [47]:
X_test_copy = X_test.copy()
X_test_grp = X_test[['matchId','groupId']].copy()
X_test_copy.drop(['matchId','groupId'], axis=1, inplace=True)
X_test_grp['winPlacePerc'] = y_pred
X_test_grp['y_pred'] = y_pred
X_test_grp.head()

,matchId,groupId,winPlacePerc,y_pred
0,45b576ab7daa7f,676b23c24e70d6,0.262560,0.262560
1,42a9a0b906c928,430933124148dd,0.918823,0.918823
2,87e7e4477a048e,0b45f5db20ba99,0.848220,0.848220
3,1b9a94f1af67f1,b7497dbdc77f4a,0.564750,0.564750
4,40754a93016066,6604ce20a1d230,0.917180,0.917180


In [48]:
# # X_test_copy.head()
# y_test['winPlacePerc_true'] = y_test['winPlacePerc']
# y_test.drop(['winPlacePerc'], axis=1, inplace=True)
# y_test.head()

In [49]:
group = X_test_grp.groupby('matchId')
X_test_grp['_rank.winPlacePerc'] = group['winPlacePerc'].rank(method='min')
print(X_test_grp[['matchId', 'groupId', 
                  '_rank.winPlacePerc', 'y_pred']].sort_values(['matchId', '_rank.winPlacePerc']).head())
X_test_copy = pd.concat([X_test_copy, X_test_grp], axis=1)
# X_test_copy = pd.merge(X_test_copy, y_test)
X_test_copy.sort_values('groupId').head()

               matchId         groupId  _rank.winPlacePerc   y_pred
86653   0008c31a9be4a7  c64735f68f511a                 1.0  0.00000
81147   0008c31a9be4a7  cb644358e14752                 2.0  0.15845
115921  0008c31a9be4a7  01fb9c20f6abc2                 3.0  0.19915
7262    0008c31a9be4a7  26d4045668cf95                 4.0  0.25688
716020  0008c31a9be4a7  44f84d3bba50e9                 5.0  0.25764


,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,...,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired,matchId,groupId,winPlacePerc,y_pred,_rank.winPlacePerc
472841,8,19,235.6,300.56,2,3,0,0,49,105,...,66.35,1642.0,4417.0,6,15,d58a74c8197fdf,00000b5b45f70c,0.75271,0.75271,23.0
344748,0,0,51.6,94.60,0,0,0,0,71,141,...,0.00,318.5,581.7,4,8,acadebc66753a6,00000fb8f2208b,0.31666,0.31666,10.0
25858,3,3,138.9,223.50,0,0,0,0,43,57,...,62.14,2917.0,4730.0,10,15,29fc53a093a0ab,0000120038fb95,0.88181,0.88181,40.0
286786,2,2,731.9,831.90,3,4,2,2,54,62,...,35.56,684.4,896.3,4,7,d6867077cbb3c6,00001e221235dd,0.60036,0.60036,29.0
645891,2,4,323.0,323.00,3,3,0,0,73,251,...,19.32,412.6,1056.2,3,9,f8821ae0e3d262,000022937e1c55,0.28443,0.28443,8.0


In [50]:
# X_test_copy[['Id', 'numGroups', 'maxPlace', 'winPlacePerc', '_rank.winPlacePerc']].head(10)
# list(train.columns)
X_test_copy.head()

,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,...,sum_longestKill,max_walkDistance,sum_walkDistance,max_weaponsAcquired,sum_weaponsAcquired,matchId,groupId,winPlacePerc,y_pred,_rank.winPlacePerc
0,0,0,74.2,125.66,1,1,0,0,73,286,...,0.000,913.0,2436.8,7,10,45b576ab7daa7f,676b23c24e70d6,0.262560,0.262560,7.0
1,6,10,597.5,776.60,4,4,1,1,11,14,...,554.500,2477.0,4494.0,7,13,42a9a0b906c928,430933124148dd,0.918823,0.918823,46.0
2,6,9,1058.0,1490.11,5,8,4,4,50,103,...,254.420,2351.0,4397.1,8,15,87e7e4477a048e,0b45f5db20ba99,0.848220,0.848220,24.0
3,0,0,100.0,165.52,1,1,0,0,54,107,...,0.000,2116.0,3928.0,5,8,1b9a94f1af67f1,b7497dbdc77f4a,0.564750,0.564750,22.0
4,4,7,330.2,661.84,2,4,2,3,23,36,...,173.838,3027.0,8695.0,9,18,40754a93016066,6604ce20a1d230,0.917180,0.917180,27.0


In [51]:
test = pd.read_csv('input/test_V2.csv')

In [52]:
X_test_copy = pd.merge(X_test_copy, test[['groupId', 'numGroups', 'maxPlace']], on='groupId')
fullgroup = (X_test_copy['numGroups'] == X_test_copy['maxPlace'])

print(sum(fullgroup))
# full group (201366) --> calculate from rank
subset = X_test_copy.loc[fullgroup]
X_test_copy.loc[fullgroup, 'winPlacePerc'] = (subset['_rank.winPlacePerc'].values - 1) / (subset['maxPlace'].values - 1)

# not full group (684872) --> align with maxPlace
subset = X_test_copy.loc[~fullgroup]
gap = 1.0 / (subset['maxPlace'].values - 1)
new_perc = np.around(subset['winPlacePerc'].values / gap) * gap  # half&up
X_test_copy.loc[~fullgroup, 'winPlacePerc'] = new_perc

X_test_copy['winPlacePerc'] = X_test_copy['winPlacePerc'].clip(lower=0,upper=1)


544654


In [53]:
print(X_test_copy.loc[~fullgroup]['matchId'].unique())
X_test_copy[['matchId', 'groupId', 'winPlacePerc', 'y_pred', 
                   '_rank.winPlacePerc']].sort_values(['matchId', 'groupId']).head(50)


['42a9a0b906c928' '87e7e4477a048e' '1b9a94f1af67f1' ... '0f498f00ef2311'
 '9fd1df68453821' 'd2cd46eed660e1']


,matchId,groupId,winPlacePerc,y_pred,_rank.winPlacePerc
353387,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0
353388,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0
353389,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0
633062,0008c31a9be4a7,0943c3f283b976,0.724138,0.816650,22.0
492359,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0
492360,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0
492361,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0
1754768,0008c31a9be4a7,1568e092a99583,0.586207,0.745250,18.0
23223,0008c31a9be4a7,26d4045668cf95,0.103448,0.256880,4.0
204885,0008c31a9be4a7,298bb0348ccd3a,0.206897,0.353470,7.0


In [54]:
X_test_copy.loc[~fullgroup, '_pred.winPlace'] = np.around(X_test_copy.loc[~fullgroup, 'winPlacePerc'].values / gap) + 1
X_test_copy.loc[~fullgroup & (X_test_copy['matchId'] == '12acd71ccf720e'),
           ['matchId','groupId','winPlacePerc','maxPlace','numGroups','_pred.winPlace','_rank.winPlacePerc']
          ].sort_values(['matchId','_pred.winPlace','_rank.winPlacePerc'])

,matchId,groupId,winPlacePerc,maxPlace,numGroups,_pred.winPlace,_rank.winPlacePerc


In [55]:
# y_test.head(), X_test_copy['winPlacePerc'].head()

In [56]:
# final_df = pd.merge(y_test, X_test_copy[['matchId', 'groupId', 'winPlacePerc', 
#                                          '_pred.winPlace', '_rank.winPlacePerc', 'y_pred']], on='groupId')
# final_df.head(50)

In [57]:
# final_df.sort_values(['matchId', 'groupId', 'winPlacePerc']).head(50)
X_test_copy.sort_values(['matchId', 'groupId', 'winPlacePerc']).head(50)

,max_boosts,sum_boosts,max_damageDealt,sum_damageDealt,max_DBNOs,sum_DBNOs,max_headshotKills,sum_headshotKills,max_killPlace,sum_killPlace,...,max_weaponsAcquired,sum_weaponsAcquired,matchId,groupId,winPlacePerc,y_pred,_rank.winPlacePerc,numGroups,maxPlace,_pred.winPlace
353387,1,1,83.00,83.00,0,0,0,0,77,228,...,3,7,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0,30,30,NaN
353388,1,1,83.00,83.00,0,0,0,0,77,228,...,3,7,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0,30,30,NaN
353389,1,1,83.00,83.00,0,0,0,0,77,228,...,3,7,0008c31a9be4a7,01fb9c20f6abc2,0.068966,0.199150,3.0,30,30,NaN
633062,6,6,104.00,104.00,1,1,0,0,22,22,...,6,6,0008c31a9be4a7,0943c3f283b976,0.724138,0.816650,22.0,30,30,NaN
492359,7,18,1261.00,2108.10,6,12,0,0,20,24,...,8,18,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0,30,30,NaN
492360,7,18,1261.00,2108.10,6,12,0,0,20,24,...,8,18,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0,30,30,NaN
492361,7,18,1261.00,2108.10,6,12,0,0,20,24,...,8,18,0008c31a9be4a7,11b26f1f710257,0.965517,0.929367,29.0,30,30,NaN
1754768,4,4,135.70,135.70,1,1,0,0,23,23,...,8,8,0008c31a9be4a7,1568e092a99583,0.586207,0.745250,18.0,30,30,NaN
23223,0,0,0.00,0.00,0,0,0,0,74,74,...,3,3,0008c31a9be4a7,26d4045668cf95,0.103448,0.256880,4.0,30,30,NaN
204885,1,1,286.00,466.41,3,4,1,1,70,241,...,5,12,0008c31a9be4a7,298bb0348ccd3a,0.206897,0.353470,7.0,30,30,NaN


In [70]:
final_df = pd.merge(test[['Id', 'groupId']], X_test_copy[['groupId', 'matchId', 'winPlacePerc', 
                                         '_pred.winPlace', '_rank.winPlacePerc', 'y_pred']], on='groupId')
final_df.drop_duplicates(inplace=True)
submission = final_df[['Id','winPlacePerc']]
submission.to_csv("submission.csv", index=False)

In [71]:
final_df.head(50)

,Id,groupId,matchId,winPlacePerc,_pred.winPlace,_rank.winPlacePerc,y_pred
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0.222222,NaN,7.0,0.262560
4,d6267a32c5709c,676b23c24e70d6,45b576ab7daa7f,0.222222,NaN,7.0,0.262560
8,b896f8954a92e2,676b23c24e70d6,45b576ab7daa7f,0.222222,NaN,7.0,0.262560
12,2f134f2c7be198,676b23c24e70d6,45b576ab7daa7f,0.222222,NaN,7.0,0.262560
16,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0.914894,44.0,46.0,0.918823
18,ef362b46754f2a,430933124148dd,42a9a0b906c928,0.914894,44.0,46.0,0.918823
20,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,0.851852,24.0,24.0,0.848220
24,77a4a76df633e3,0b45f5db20ba99,87e7e4477a048e,0.851852,24.0,24.0,0.848220
28,bfe3f0a1c67d92,0b45f5db20ba99,87e7e4477a048e,0.851852,24.0,24.0,0.848220
32,9b5a556b0c8b30,0b45f5db20ba99,87e7e4477a048e,0.851852,24.0,24.0,0.848220


In [72]:
final_df.shape, X_test_copy.shape, test.shape, len(final_df['Id'].unique())

((1934174, 7), (1934174, 26), (1934174, 28), 1934174)

# Result

Attributes | MAE
--- | ---
**Decision Tree** |
All | 0.0822435372194249
All with Ranking | 0.020896060576532312, 0.10
All with Ranking (with ascending False) | 0.19069269005752446
Features from Chicken Dinner Notebook (including maxplace, numgroups) | 0.024114451660075895 
Features from Chicken Dinner Notebook | 0.11921477140699023, 0.11922039296180897
**Random Forest** |
All with Ranking | 0.0004156307577606391
Features from Chicken Dinner Notebook | 0.09820843398398921

wow: 0.026739255251196675
0.0280428038675312

(TODO) <br>
Because how dynamic a winPlacePerc value can be, custom prediction function needs to be developed

In [44]:
# final_df[['Id', 'winPlacePerc']]